In [1]:
import pandas as pd

df = pd.read_parquet('dataset_permissive.parquet')

In [2]:
len(df)

18162

In [3]:
df.head()

,entry_point,original_triton_code,python_code,triton_code,repo_name,module_name,synthetic,uuid,licenses,stars,sha,repo_link
0,SumAggregator,# AOT ID: ['0_inference']\nfrom ctypes import ...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport triton\nimport triton.lan...,AlexMinhao/NAS_GNN,SumAggregator,False,0,[Apache-2.0],0,89183988a96e1d6baed910ab3843c13282f8b077,https://github.com/AlexMinhao/NAS_GNN/tree/891...
1,LinearEmbedding,# AOT ID: ['0_forward']\nfrom ctypes import c_...,import math\nimport torch\nimport torch.utils....,import torch\nfrom torch._inductor.select_algo...,Akhil-Raj/Trajectory-Transformer,LinearEmbedding,False,1,[MIT],0,dd09fda99443f6afb59d962026573162219ea6a9,https://github.com/Akhil-Raj/Trajectory-Transf...
2,CustomizeLayer,# AOT ID: ['0_forward']\nfrom ctypes import c_...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport triton\nimport triton.lan...,Abhishekvats1997/Torch-Pruning,CustomizeLayer,False,2,[MIT],0,b322a42d1c9032cc9644332d33a9662ca6ed44ac,https://github.com/Abhishekvats1997/Torch-Prun...
3,LayerNorm,# AOT ID: ['0_forward']\nfrom ctypes import c_...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport triton\nimport triton.lan...,AWilcke/Dissertation,LayerNorm,False,3,[MIT],0,b85ad38a7f336ee290d5883f5e942f54e140d0d0,https://github.com/AWilcke/Dissertation/tree/b...
4,LayerNorm,# AOT ID: ['0_forward']\nfrom ctypes import c_...,import torch\nimport torch.utils.data\nimport ...,import torch\nimport triton\nimport triton.lan...,Akhil-Raj/Trajectory-Transformer,LayerNorm,False,4,[MIT],0,dd09fda99443f6afb59d962026573162219ea6a9,https://github.com/Akhil-Raj/Trajectory-Transf...


In [4]:
df.iloc[348]

entry_point                                                     AngleLoss
original_triton_code    # AOT ID: ['0_inference']\nfrom ctypes import ...
python_code             import torch\nfrom torch import nn\nimport tor...
triton_code             import torch\nimport triton\nimport triton.lan...
repo_name                                    DeVriesMatt/pointMLP-pytorch
module_name                                                     AngleLoss
synthetic                                                           False
uuid                                                                  365
licenses                                                     [Apache-2.0]
stars                                                                   0
sha                              e9c09a2038551e83b072353f3fd7e3294463e892
repo_link               https://github.com/DeVriesMatt/pointMLP-pytorc...
Name: 348, dtype: object

In [5]:
example_row = df.iloc[348]
print(example_row['python_code'])

import torch
from torch import nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
from typing import *


class AngleLoss(nn.Module):

    def __init__(self):
        super(AngleLoss, self).__init__()

    def forward(self, angle, angle_hat):
        return torch.exp(F.mse_loss(angle_hat.float(), angle.float())) - 1


def get_inputs():
    return [torch.rand([4, 4, 4, 4]), torch.rand([4, 4, 4, 4])]


def get_init_inputs():
    return [[], {}]



In [6]:
import re
# replace the entry in the python code with "Model"
def fix_python_code(code, entry_point):
    model_pattern = f"class {entry_point}(nn.Module):"
    super_pattern = f"super({entry_point})"
    
    # assert that each pattern appears only once in the code
    assert code.count(model_pattern) == 1, f"Model pattern must appear only once in the code: {model_pattern}"
    assert code.count(super_pattern) <= 1, f"Super pattern must appear only once in the code: {super_pattern}"
    
    code = re.sub(model_pattern, "class Model(nn.Module):", code)
    code = re.sub(super_pattern, "super(Model)", code)

    return code

print(fix_python_code(example_row['python_code'], example_row['entry_point']))

import torch
from torch import nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
from typing import *


class AngleLoss(nn.Module):

    def __init__(self):
        super(AngleLoss, self).__init__()

    def forward(self, angle, angle_hat):
        return torch.exp(F.mse_loss(angle_hat.float(), angle.float())) - 1


def get_inputs():
    return [torch.rand([4, 4, 4, 4]), torch.rand([4, 4, 4, 4])]


def get_init_inputs():
    return [[], {}]



In [7]:
import os 
ref_arch_path = "KernelBench/KernelBench/level9/54_Linear_soft_plus.py"
os.path.basename(ref_arch_path)

'54_Linear_soft_plus.py'

In [8]:
problem_name = "54_Linear_soft_plus.py"
name = problem_name.replace(".py", "")
first_underscore = name.find("_")
entry_point = name[first_underscore+1:]
print(entry_point)

Linear_soft_plus


In [9]:
# # Loop through each row in the dataframe and write the python code to a file
# import os
# from tqdm import tqdm

# output_dir = '../KernelBench/level9'
# problem_id = 1

# for index, row in tqdm(df.iterrows(), total=len(df)):
#     entry_point = row['entry_point']
#     python_code = row['python_code']
#     file_name = f"{problem_id}_{entry_point}.py"
#     problem_id += 1
    
#     assert "def get_init_inputs():\n    return [[]" in python_code, f"get_init_inputs() not found in {file_name}"
#     # with open(os.path.join(output_dir, file_name), 'w') as f:
#     #     f.write(python_code)